In [1]:
import os
import cv2
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split
from IPython.display import clear_output

In [45]:
actions_csv_dir = '../landmark-csv-processed/'
dataset = []
actions = ['swing', 'slide', 'sit', 'lie', 'backward', 'walk']
label_mapping = {'swing': 0,
    'slide': 1,
    'sit': 2,
    'lie': 3,
    'backward': 4,
    'walk': 5
}
length = 20 # 그리고 데이터를 LSTM에 넣어야 하기 때문에 시퀀스로 묶어주어야 한다. 그 길이가 length라는 길이이고, 이 변수를 20으로 설정했다. 20의 의미는 20개의 frame을 하나의 시퀀스로 잡아서 입력으로 넣겠다는 의미이고,....
# 영상 최소 길이로 묶어야하나? 아닌데 그냥 적당히 자르면 되겠지..근데 동작마다 끝나는 시점이 다른데 뭘로 묶냐
def map_action_to_label(csv_name):
    for action, label in label_mapping.items():
        if action in csv_name:
            return label
    return -1

for action_csv in os.listdir(actions_csv_dir):
    action_df = pd.read_csv(os.path.join(actions_csv_dir, action_csv)) #실제로 csv가 아니라 list일걸요
    label = map_action_to_label(action_csv)
    if label != -1:
        for idx in range(0, len(action_df), int(length / 2)):
            seq_df = action_df[idx: idx + length] #길이만큼 데이터 자른 것(즉 length 만큼의 프레임)
            if len(seq_df) == length: # 딱 length에 개수 맞춰서 끊어서 넣으려고
                dataset.append({'key': label, 'value': seq_df}) # key에 slide, value에는 묶음 프레임 만큼이 담기겠네
    #최종적으로 dataset에는 행동별로 dictionary 가 만들어져 들어간다.

AttributeError: 'list' object has no attribute 'keys'

In [40]:
print(dataset[0]['value'].columns)

Index(['Nose_x', 'Nose_y', 'Neck_x', 'Neck_y', 'RShoulder_x', 'RShoulder_y',
       'RElbow_x', 'RElbow_y', 'RWrist_x', 'RWrist_y', 'LShoulder_x',
       'LShoulder_y', 'LElbow_x', 'LElbow_y', 'LWrist_x', 'LWrist_y',
       'MidHip_x', 'MidHip_y', 'RHip_x', 'RHip_y', 'RKnee_x', 'RKnee_y',
       'RAnkle_x', 'RAnkle_y', 'LHip_x', 'LHip_y', 'LKnee_x', 'LKnee_y',
       'LAnkle_x', 'LAnkle_y', 'REye_x', 'REye_y', 'LEye_x', 'LEye_y',
       'REar_x', 'REar_y', 'LEar_x', 'LEar_y', 'LBigToe_x', 'LBigToe_y',
       'LSmallToe_x', 'LSmallToe_y', 'LHeel_x', 'LHeel_y', 'RBigToe_x',
       'RBigToe_y', 'RSmallToe_x', 'RSmallToe_y', 'RHeel_x', 'RHeel_y'],
      dtype='object')


In [41]:
if torch.cuda.is_available() == True:
    device = 'cuda:0'
    print('현재 가상환경 GPU 사용 가능 상태')
else:
    device = 'cpu'
    print('GPU 사용 불가능 상태')

GPU 사용 불가능 상태


In [46]:
class MyDataset(Dataset):
    def __init__(self, seq_data): #모든 행동을 통합한 df가 들어가야함
        self.x = []
        self.y = []
        for dic in seq_data:
            self.y.append(dic['key']) #key 값에는 actions 들어감
            self.x.append(dic['value']) #action마다의 data 들어감

    def __getitem__(self, index): #index는 행동의 index
        data = self.x[index] # x에는 꺼내 쓸 (행동마다 20개 묶음프레임)의 데이터
        label = self.y[index]
        return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))

    def __len__(self):
        return len(self.x)


In [47]:
train_test_val_ratio = [0.8, 0.1, 0.1]
print(len(dataset))
train_len = int(len(dataset) * train_test_val_ratio[0])
val_len = int(len(dataset) * train_test_val_ratio[1])
test_len = len(dataset) - train_len - val_len
print('{}, {}, {}'.format(train_len, val_len, test_len))

40
32, 4, 4


In [51]:
train_dataset = MyDataset(dataset)
train_data, valid_data, test_data = random_split(train_dataset, [train_len, val_len, test_len])

train_loader = DataLoader(train_data, batch_size=8)
val_loader = DataLoader(valid_data, batch_size=8)
test_loader = DataLoader(test_data, batch_size=8)

In [52]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.lstm1 = nn.LSTM(input_size=135, hidde_size=128, num_layers=1, batch_first=True) #input은  45 * 3(x, y z)
        self.lstm2 = nn.LSTM(input_size=128, hidde_size=256, num_layers=1, batch_first=True)
        self.lstm3 = nn.LSTM(input_size=256, hidde_size=512, num_layers=1, batch_first=True)
        self.dropout1 = nn.Dropout(0, 1)
        self.lstm4 = nn.LSTM(input_size=512, hidde_size=256, num_layers=1, batch_first=True)
        self.lstm5 = nn.LSTM(input_size=256, hidde_size=128, num_layers=1, batch_first=True)
        self.lstm6 = nn.LSTM(input_size=128, hidde_size=64, num_layers=1, batch_first=True)
        self.dropout2 = nn.Dropout(0, 1)
        self.lstm7 = nn.LSTM(input_size=64, hidde_size=32, num_layers=1, batch_first=True)
        self.fc = nn.Linear(32, 5) #분류할 클래스 5가지

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.dropout1(x)
        x, _ = self.lstm4(x)
        x, _ = self.lstm5(x)
        x, _ = self.lstm6(x)
        x = self.dropout2(x)
        x, _ = self.lstm7(x)
        x = self.fc(x[:, -1, :])
        return x

In [55]:
def init_model():
    plt.rc('font', size=10)
    global net, loss_fn, optim
    net = Model().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optim = Adam(net.parameters(), lr=0.0001)

def init_epoch():
    global epoch_cnt
    epoch_cnt = 0

def init_log():
    plt.rc('font', size=10)
    global log_stack, iter_log, tloss_log, tacc_log, vloss_log, vacc_log, time_log
    iter_log, tloss_log, tacc_log, vloss_log, vacc_log = [], [], [], [], []
    time_log, log_stack = [], []

def record_train_log(_tloss, _tacc, _time):
    # train log 기록
    time_log.append(_time)
    tloss_log.append(_tloss)
    tacc_log.append(_tacc)
    iter_log.append(epoch_cnt)


def record_valid_log(_vloss, _vacc):
    # validation log 기록
    vloss_log.append(_vloss)
    vacc_log.append(_vacc)


def last(log_list):
    # 리스트 안의 마지막 숫자를 반환 -> print_log함수에서 사용
    if len(log_list) > 0:
        return log_list[len(log_list) - 1]
    else:
        return -1

def print_log():
    train_loss = round(float(last(tloss_log)), 3)
    train_acc = round(float(last(tacc_log)), 3)
    val_loss = round(float(last(vloss_log)), 3)
    val_acc = round(float(last(vacc_log)), 3)
    time_spent = round(float(last(time_log)), 3)

    log_str = 'Epoch: {:3} | T_Loss {:5} | T_acc {:5} | V_Loss {:5} | V_acc. {:5} | \ {:5}'.format(last(iter_log),train_loss, train_acc, val_loss, val_acc, time_spent)
    log_stack.append(log_str)

    # 학습 추이 그래프 출력
    hist_fig, loss_axis = plt.subplots(figsize=(10, 3), dpj=99)
    hist_fig.patch.set_facecolor('white')

    # Loss Line 구성
    loss_t_line = plt.plot(iter_log, tloss_log, label='Train Loss', color='red', marker='o')
    loss_v_line = plt.plot(iter_log, vloss_log, label='Valid Loss', color='blue', marker='s')
    loss_axis.set_xlabel('epoch')
    loss_axis.set_ylabel('loss')

    # ACC, Line 구성
    acc_axis = loss_axis.twinx()
    acc_t_line = acc_axis.plot(iter_log, tacc_log, label='Train Acc', color='red', marker='+')
    acc_v_line = acc_axis.plot(iter_log, vacc_log, label='Valid Acc', color='blue', marker='x')
    acc_axis.set_ylabel('accuracy')

    # 그래프 출력
    hist_lines = loss_t_line + loss_v_line + acc_t_line + acc_v_line
    loss_axis.legend(hist_lines, [i.get_label() for i in hist_lines])
    loss_axis.grid()
    plt.title('Learning history until epoch {}'.format(last(iter_log)))
    plt.draw()

    # 텍스트 로그 출력
    clear_output(wait=True)
    plt.show()
    for idx in reversed(range(len(log_stack))):
        print(log_stack[idx])


In [56]:
import gc
from torch.cuda import empty_cache


def clear_memory():
    if device != 'cpu':
        empty_cache()
    gc.collect()

In [ ]:
def epoch(data_loader, mode='train'):
    global epoch_cnt

    # 변수 초기화
    iter_loss, iter_acc, last_grad_performed = [], [], False
    # 1 iteration 학습 알고리즘 (for문 1번 -> 1 epoch 완료)
    for _data, _label in data_loader:
        data, label = _data.to(device), _label.type(torch.LongTensor).to(device)

        # 1. Feed forward
        if mode == 'train':
            net.train()
        else:
            net.eval()

        result = net(data)
        _, out = torch.max(result, 1)

        # 2.Loss 계산
        loss = loss_fn(result, label)
        iter_loss.append(loss.item())

        # 3.역전파 학습 후 Gradient Descent
        if mode == 'train':
            optim.zero_grad()
            loss.backward()
            optim.step()
            last_grad_performed = True

        # 4. 정확도 계산
        acc_partial = (out == label).float().sum()
        acc_partial = acc_partial / len(label)
        iter.acc.append(acc_partial.item)

    # 역잔사 학습 후, Epoch 카운터 += 1
    if last_grad_performed:
        epoch_cnt += 1

    clear_memory

    return np.average(iter_loss), np.average(iter_acc)


In [ ]:
maximum_epoch = 10

def epoch_not_finished():
    # 에폭이 끝남음을 알림
    return epoch_cnt < maximum_epoch

# 정확도 검증
with torch.no_grad():
    test_loss, test_acc = epoch(test_loader, mode='test')
    test_acc = round(test_acc, 4)
    test_loss = round(test_loss, 4)
    print('Test Acc.: {}'.format(test_acc))
    print('Test Loss: {}'.format(test_loss))


In [ ]:
# 여기부터 실시간 영상 테스트

interval = 1
video_path = './tataset/day4_lstm/test_data/test_video.mp4'
cv2.destroyAllWindows()
cap = cv2.VideoCapture(video_path)
img_list = []
if cap.isOpened():
    cnt = 0
    while True:
        ret, img = cap.read()
        if ret:
            img = cv2.resize(img, (640, 640))
            if cnt == interval:
                img_list.append(img)
                cnt = 0
            cv2.imshow(video_path, img)
            cv2.waitKey(1)
            cnt += 1
        else:
            break
cap.release()
cv2.destroyAllWindows()

print('저정된 frame의 개수: {}'.format(len(img_list)))

In [ ]:
# 연속 시퀀스 분석
from tqdm import tqdm

net.eval()
out_img_list = []
dataset = []
status = 'None'
pose = np.pose.Pose(static_image_mode=True, model_complexity=1, enable_segmentaion=False, min_detection_confidence=0.3)

print('시퀀스 데이터 분석 중...')
xy_list_list = []

if len(xy_list_list) == length:
    dataset = []
    dataset.append({'key': 0, 'value': xy_list_list})
    dataset = MyDataset(dataset)
    dataset = DataLoader(dataset)
    xy_list_list = []
    for data, label in dataset:
        data = data.to(device)
        with torch.no_grad():
            result = net(data)
            _, out = torch.max(result, 1)
            if out.item() == 0:
                status = 'Walking'
            else:
                status = 'Running'
cv2.putText(img, status, (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0, 0, 255), 2)
out_img_list.append(img)